In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

In [2]:
with open('./pickle/test_complete_df_6개월_10개.pkl', 'rb') as f:
    stock_df = pickle.load(f)
with open('./pickle/6개월_10개_cor_0.075_lstm.pkl', 'rb') as f:
    col_list = pickle.load(f)

In [3]:
stock_df

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,헝가리BUX내역_고가,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,호주S&PASX내역_종가,호주S&PASX내역_오픈,호주S&PASX내역_고가,호주S&PASX내역_저가,호주S&PASX내역_거래량,호주S&PASX내역_변동
0,20220502,901,12450.0,12550.0,12450.0,12550.0,3049.0,37980000.0,3049.0,0.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
1,20220502,902,12500.0,12600.0,12500.0,12500.0,662.0,8300000.0,3539.0,172.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
2,20220502,903,12550.0,12600.0,12500.0,12600.0,392.0,4920000.0,3890.0,213.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
3,20220502,904,12600.0,12600.0,12550.0,12600.0,1092.0,13760000.0,4700.0,495.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
4,20220502,905,12600.0,12600.0,12600.0,12600.0,113.0,1420000.0,4813.0,495.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52573,20221121,1517,2060.0,2070.0,2060.0,2070.0,0.0,0.0,24646.0,49781.0,...,44778.83,44034.23,0,0.22,7139.3,7151.8,7174.4,7131.7,598630000.0,-0.17
52574,20221121,1518,2070.0,2070.0,2070.0,2070.0,211.0,440000.0,24857.0,49781.0,...,44778.83,44034.23,0,0.22,7139.3,7151.8,7174.4,7131.7,598630000.0,-0.17
52575,20221121,1519,2070.0,2070.0,2070.0,2070.0,0.0,0.0,24857.0,49781.0,...,44778.83,44034.23,0,0.22,7139.3,7151.8,7174.4,7131.7,598630000.0,-0.17
52576,20221121,1520,2070.0,2070.0,2070.0,2070.0,0.0,0.0,24857.0,49781.0,...,44778.83,44034.23,0,0.22,7139.3,7151.8,7174.4,7131.7,598630000.0,-0.17


In [4]:
# corr_matrix = stock_df.corr()
# cor = corr_matrix["pct_label"]
# cor1 = cor[(cor.values>0.075) | (cor.values<-0.075)]
#         # | (cor.index == '고가') | (cor.index == '시가') | (cor.index == '종가') | (cor.index == '저가')  ]

# cor1 = cor1.drop('pct_label')  # 고가, 종가, 시가, 저가
# len(cor1)
# # cor1.to_pickle('./pickle/6개월_10개_cor_0.12_강화학습.pkl')

C:\Users\USER\AppData\Local\Temp\ipykernel_3168\2548490762.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = stock_df.corr()


In [100]:
stock_df['pct_label'] = stock_df['pct_label'].apply(np.floor)
cate = pd.DataFrame({'pct_label' : [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
                                    ,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]})

ohe = OneHotEncoder(sparse=False)
ohe_test = ohe.fit_transform(cate[['pct_label']])
# ohe.categories_
len(ohe_test[0])
# pd.DataFrame(ohe_test, columns=['pct_label_' + str(col) for col in ohe.categories_[0]])

In [104]:
y_stock_df = ohe.transform(stock_df[['pct_label']])
y_stock_df.shape

(525780, 61)

In [109]:
col_list = list(cor1.index)
X_stock_df = stock_df.drop(['pct_label'],axis=1)
X_stock_df = X_stock_df.set_index(['날짜'])
X_stock_df = X_stock_df[col_list]
X_stock_df

,시간,시가,고가,저가,종가,전일대비,AEDKRW내역_오픈,AEDKRW내역_고가,AEDKRW내역_저가,ATX내역_종가,...,한국5년채권수익율내역_고가,한국5년채권수익율내역_저가,헝가리BUX내역_종가,헝가리BUX내역_오픈,헝가리BUX내역_고가,헝가리BUX내역_저가,호주S&PASX내역_종가,호주S&PASX내역_오픈,호주S&PASX내역_고가,호주S&PASX내역_저가
날짜,,,,,,,,,,,,,,,,,,,,,
20220502,901,12450.0,12550.0,12450.0,12550.0,-100,344.01,345.96,343.82,3236.28,...,3.322,3.261,42703.38,43465.06,43468.19,42571.10,7347.0,7435.0,7435.0,7301.6
20220502,902,12500.0,12600.0,12500.0,12500.0,-100,344.01,345.96,343.82,3236.28,...,3.322,3.261,42703.38,43465.06,43468.19,42571.10,7347.0,7435.0,7435.0,7301.6
20220502,903,12550.0,12600.0,12500.0,12600.0,-100,344.01,345.96,343.82,3236.28,...,3.322,3.261,42703.38,43465.06,43468.19,42571.10,7347.0,7435.0,7435.0,7301.6
20220502,904,12600.0,12600.0,12550.0,12600.0,-100,344.01,345.96,343.82,3236.28,...,3.322,3.261,42703.38,43465.06,43468.19,42571.10,7347.0,7435.0,7435.0,7301.6
20220502,905,12600.0,12600.0,12600.0,12600.0,-100,344.01,345.96,343.82,3236.28,...,3.322,3.261,42703.38,43465.06,43468.19,42571.10,7347.0,7435.0,7435.0,7301.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221121,1517,2060.0,2070.0,2060.0,2070.0,-25,364.82,370.94,364.82,3210.26,...,3.877,3.830,44592.90,44496.51,44778.83,44034.23,7139.3,7151.8,7174.4,7131.7
20221121,1518,2070.0,2070.0,2070.0,2070.0,-25,364.82,370.94,364.82,3210.26,...,3.877,3.830,44592.90,44496.51,44778.83,44034.23,7139.3,7151.8,7174.4,7131.7
20221121,1519,2070.0,2070.0,2070.0,2070.0,-25,364.82,370.94,364.82,3210.26,...,3.877,3.830,44592.90,44496.51,44778.83,44034.23,7139.3,7151.8,7174.4,7131.7


In [110]:
X_stock_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 525780 entries, 20220502 to 20221121
Columns: 401 entries, 시간 to 호주S&PASX내역_저가
dtypes: float64(393), int64(8)
memory usage: 1.6 GB


In [111]:
min_max_scaler = MinMaxScaler()

X_stock_sc = min_max_scaler.fit_transform(X_stock_df)

In [ ]:
# X_stock_trans_df = pd.DataFrame(data=X_stock_sc, columns=X_stock_df.columns)
# X_stock_df = X_stock_trans_df.values.tolist()

In [ ]:
# window_size = 10

# data_X = []
# data_y = []
# for i in range(len(y_stock_df) - window_size):
#     _X = X_stock_df[i : i + window_size] # 다음 날 종가(i+windows_size)는 포함되지 않음
#     _y = y_stock_df[i + window_size]     # 다음 날 종가
#     data_X.append(_X)
#     data_y.append(_y)
# print(_X, "->", _y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_stock_sc, y_stock_df, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1295, 1)
X_test = X_test.reshape(X_test.shape[0], 1295, 1)
print('훈련 데이터의 크기 :', X_train.shape, y_train.shape)
print('테스트 데이터의 크기 :', X_test.shape, y_test.shape)

훈련 데이터의 크기 : (381379, 1295, 1) (381379, 65)
테스트 데이터의 크기 : (163449, 1295, 1) (163449, 65)


In [ ]:
# train_size = int(len(data_y) * 0.7)
# X_train = np.array(data_X[0 : train_size])
# y_train = np.array(data_y[0 : train_size])

# test_size = len(data_y) - train_size
# X_test = np.array(data_X[train_size : len(data_X)])
# y_test = np.array(data_y[train_size : len(data_y)])

# print('훈련 데이터의 크기 :', X_train.shape, y_train.shape)
# print('테스트 데이터의 크기 :', X_test.shape, y_test.shape)

In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout, PReLU, LeakyReLU, Input, BatchNormalization, Activation, Flatten
from tensorflow.keras.activations import relu, tanh
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers


In [ ]:
inputs = Input(shape=(1295, 1))
lstm_out = LSTM(244, dropout=0.2, return_sequences=True)(inputs)
lstm_out = BatchNormalization()(lstm_out)
lstm_out = Activation('LeakyReLU')(lstm_out)
lstm_out = LSTM(122, dropout=0.2)(lstm_out)
lstm_out = BatchNormalization()(lstm_out)
lstm_out = Activation('LeakyReLU')(lstm_out)
lstm_out = Dense(65)(lstm_out)
lstm_out = BatchNormalization()(lstm_out)
lstm_out = Activation('softmax')(lstm_out)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)
# 모델의 이름을 정합니다.
modelpath="./model/lstm_test.hdf5"

# 최적화 모델을 업데이트하고 저장합니다.
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

model = Model(inputs=inputs, outputs=lstm_out)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001, decay=1e-7), metrics=['accuracy'])

history=model.fit(X_train, y_train, epochs=10, batch_size=1024,verbose=1, validation_data=(X_test, y_test), callbacks=[early_stopping_callback, checkpointer])


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1295, 1)]         0         
                                                                 
 lstm (LSTM)                 (None, 1295, 244)         240096    
                                                                 
 batch_normalization (BatchN  (None, 1295, 244)        976       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1295, 244)         0         
                                                                 
 lstm_1 (LSTM)               (None, 122)               179096    
                                                                 
 batch_normalization_1 (Batc  (None, 122)              488       
 hNormalization)                                             

KeyboardInterrupt: 

In [ ]:
model.load_weights(modelpath)

In [ ]:
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

1821/1821 [==============================] - 38s 21ms/step - loss: 1.2864 - accuracy: 0.5578

 Test Accuracy: 0.5578


In [ ]:
predict = model.predict(X_test)

1821/1821 [==============================] - 40s 22ms/step


In [ ]:
pred_df = pd.read_csv('./temp_data/test_complete_A066970.csv')

In [ ]:
X_pred_df = pred_df.iloc[:,0:-1]
pred_df['pct_label'] = pred_df['pct_label'].apply(np.floor)
y_pred_df = ohe.transform(pred_df[['pct_label']])
y_pred_df.shape

(1905, 61)

In [ ]:
X_pred_sc = min_max_scaler.fit_transform(X_pred_df)
X_pred = X_pred_sc.reshape(X_pred_sc.shape[0], 21, 1)
X_pred.shape

(1905, 21, 1)

In [ ]:
predict = model.predict(X_pred)

60/60 [==============================] - 1s 20ms/step


(1905, 61)

In [ ]:
import tensorflow as tf
pred = tf.argmax(predict,1).numpy()
test = tf.argmax(y_pred_df,1).numpy()

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(test, pred)
print('accuracy_score : ',acc)

accuracy_score :  0.48241469816272964
